# **Reproducing Figure 2**

<a target="_blank" href="https://colab.research.google.com/github/blackswan-advitamaeternam/HVAE/blob/raph/paper_experiments/Figure2_exp.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a>

## **Colab setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# to avoid having the data on your drive
%cd /content

Mounted at /content/drive
/content


In [ ]:
!git clone https://github.com/blackswan-advitamaeternam/HVAE.git
%cd HVAE
!git checkout raph
!pip install -r requirements.txt

Cloning into 'HVAE'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 354 (delta 6), reused 8 (delta 3), pack-reused 317 (from 1)
Receiving objects: 100% (354/354), 276.30 KiB | 641.00 KiB/s, done.
Resolving deltas: 100% (228/228), done.
/content/HVAE
Branch 'raph' set up to track remote branch 'raph' from 'origin'.
Switched to a new branch 'raph'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 102.3 MB/s eta 0:00:00


To allow automatic reloading

In [ ]:
!pip install --upgrade ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 621.4/621.4 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.7.1
    Uninstalling traitlets-5.7.1:
      Successfully uninstalled traitlets-5.7.1
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 9.8.0 which is incompatible.


In [ ]:
import sys
try:
    import imp
except ImportError:
    import types
    sys.modules['imp'] = types.ModuleType('imp')

In [ ]:
%load_ext autoreload
%autoreload 2

## **Imports**

In [ ]:
import sys, os
# To ensure the custom package is found
path_to_repo = "/content/HVAE"
if path_to_repo not in sys.path:
    sys.path.append(path_to_repo)

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from svae.vae import SVAE, GaussianVAE
from svae.training import training
from svae.utils import ShuffledLoader

from paper_experiments.load_MNIST import make_splits_loaders_MNIST

/content/HVAE/paper_experiments/load_MNIST.py
/content/HVAE/paper_experiments


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# make splits
NUM_WORKERS = int(0.8*os.cpu_count())
FRAC = 1
TRAIN_FRAC = int(50000 * FRAC)
VAL_FRAC = int(10000 * FRAC)
TEST_FRAC = None # potentially still test on all test samples
print(f"Using {NUM_WORKERS} workers.")
train_loader, val_loader, test_loader = make_splits_loaders_MNIST(train_size=TRAIN_FRAC, val_size=VAL_FRAC, test_size=TEST_FRAC,
                                                                batch_size=100,
                                                                test_batch_size=100,
                                                                num_workers=NUM_WORKERS,
                                                                prefetch_factor=2,
                                                                force=True,
                                                                persistent_workers=True,
                                                                pin_memory=True)

Using 1 workers.

Making splits..


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.QR60UG_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.QR60UG_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
600
torch.Size([60000, 784])
100
torch.Size([10000, 784])
Train 5000
Val 1000
Test 10000


In [ ]:
# Manifesting to memory of the device (we can afford it on colab ?)
train_batches = [[el.to(DEVICE) for el in batch] for batch in train_loader]
val_batches   = [[el.to(DEVICE) for el in batch] for batch in val_loader]
test_batches  = [[el.to(DEVICE) for el in batch] for batch in test_loader]

# Wrap for shuffling behavior
train_loader = ShuffledLoader(
    train_batches,
    shuffle_batches=True,
    shuffle_within_batch=True,
    device_for_randperm=DEVICE)

val_loader = val_batches
test_loader = test_batches

## **Configuration**

In [ ]:
base_path = "/content/drive/MyDrive/HVAE/Figure2/"
os.makedirs(base_path, exist_ok=True)

In [ ]:
EPOCHS = 500
INPUT_DIM = 784
HIDDEN_DIM = 128
LATENT = 5

LATENT_MODE = 'sample'
PATIENCE = 50
WARMUP = 100
ONE_LAYER = False
LR = 1e-3

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

def get_latent_representations(model, loader, device):
    model.eval()
    latents = []
    labels = []
    
    with torch.no_grad():
        for batch in loader:
            # Depending on your specific Loader wrapper, batch might be a list or tuple.
            # Assuming standard structure [data, labels]
            x = batch[0].to(device)
            y = batch[1].to(device)
            
            # Access latent samples as specified
            # svae_latent_samples, _, _ = model.get_latent_samples(data_tensor)
            z, _, _ = model.get_latent_samples(x)
            
            latents.append(z.cpu().numpy())
            labels.append(y.cpu().numpy())
            
    return np.concatenate(latents), np.concatenate(labels)

In [ ]:
def plot_hammer_projection_S2(latents, labels, save_path):
    """
    Plots the S2 latent space using the Hammer projection to resemble Figure 2b.
    """
    # 1. Normalize vectors (Project onto unit sphere)
    norms = np.linalg.norm(latents, axis=1, keepdims=True)
    latents = latents / (norms + 1e-8)
    
    x_coord, y_coord, z_coord = latents[:, 0], latents[:, 1], latents[:, 2]
    
    # 2. Convert Cartesian to Spherical coordinates for Hammer projection
    # Longitude (lambda): [-pi, pi], Latitude (phi): [-pi/2, pi/2]
    longitude = np.arctan2(y_coord, x_coord) 
    latitude = np.arcsin(z_coord) 
    
    # 3. Setup Plot
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='hammer')
    
    # Colors for the 10 MNIST digits
    colors = plt.cm.tab10(np.linspace(0, 1, 10))
    
    # Scatter plot per digit
    for i in range(10):
        idx = (labels == i)
        if np.sum(idx) > 0:
            ax.scatter(longitude[idx], latitude[idx], 
                       label=str(i), 
                       s=8,           # Small point size like the paper
                       alpha=0.7,     # Slight transparency
                       color=colors[i],
                       edgecolors='none')
            
    # 4. Remove Axes and Grids to match Figure 2 style
    ax.grid(False)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    # We keep the spinal border to show the Hammer oval shape, 
    # but remove the ticks.
    ax.tick_params(axis='both', which='both', length=0)
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Digit", frameon=False)
    plt.title("S-VAE Latent Space ($S^2$ Hammer Projection)")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()


def plot_R2(latents, labels, save_path):
    """
    Plots the R2 latent space to resemble Figure 2a.
    """
    plt.figure(figsize=(8, 8))
    
    colors = plt.cm.tab10(np.linspace(0, 1, 10))
    
    for i in range(10):
        idx = (labels == i)
        if np.sum(idx) > 0:
            plt.scatter(latents[idx, 0], latents[idx, 1], 
                        label=str(i), 
                        s=8, 
                        alpha=0.7, 
                        color=colors[i],
                        edgecolors='none')
            
    # Remove all axes for the clean "cloud" look seen in Figure 2a
    plt.axis('off')
    
    plt.legend(title="Digit", frameon=False)
    plt.title("N-VAE Latent Space ($\mathbb{R}^2$)")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
print(f"\n[SVAE] Instantiating SVAE and optimizer..")
model = SVAE(INPUT_DIM,
            HIDDEN_DIM,
            3, # S^2
            ONE_LAYER,
            mode='MNIST')
# To device
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

print(f"[SVAE] Started training..")
model, losses, all_parts = training(train_loader,
                                val_loader,
                                model,
                                optimizer,
                                epochs=EPOCHS,
                                beta_kl=1,
                                warmup=WARMUP,
                                patience=PATIENCE,
                                show_loss_every=1,
                                )

In [ ]:
print("Extracting SVAE representations...")
# Ideally, pass the SVAE model object here
z_svae, y_svae = get_latent_representations(model, test_loader, DEVICE)

svae_save_path = os.path.join(base_path, "figure2_svae_hammer.pdf")
plot_hammer_projection_S2(z_svae, y_svae, svae_save_path)
print(f"SVAE plot saved to {svae_save_path}")

In [ ]:
print(f"\n[NVAE] Instantiating NVAE and optimizer..")
model = GaussianVAE(INPUT_DIM,
            HIDDEN_DIM,
            2, # R^2
            ONE_LAYER,
            mode='MNIST')
# To device
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

print(f"[NVAE] Started training..")
model, losses, all_parts = training(train_loader,
                                val_loader,
                                model,
                                optimizer,
                                epochs=EPOCHS,
                                beta_kl=1,
                                warmup=WARMUP,
                                patience=PATIENCE,
                                show_loss_every=1,
                                )

In [ ]:
print("Extracting NVAE representations...")
z_nvae, y_nvae = get_latent_representations(model, test_loader, DEVICE)

nvae_save_path = os.path.join(base_path, "figure2_nvae_R2.pdf")
plot_R2(z_nvae, y_nvae, nvae_save_path)
print(f"NVAE plot saved to {nvae_save_path}")